# Naive Bayes

In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

In [2]:
# Loading a dataset
X, y = load_iris(return_X_y=True)

A naive Bayes classifier tries to model $p(y | x)$ by making the naive assumption that all features are independent. With Bayes' theorem we can rewrite the distribution over classes as

$$p(y | x) = \frac{p(x | y) p(y)}{p(x)}$$

where $p(x | y)$ can be factorized as $\prod_i p(x_i | y, x_1, \dots, x_{i - 1})$. The independence assumption simplifies this expression to $\prod_i p(x_i | y)$. So for categorical $y$ we can fit a a naive Bayes model by using the proportions of classes in the training data as the prior $p(y)$ and fitting a model $p(x_i | y)$ to each feature for each class. This can most definitely be derived by the maximum likelihood principle.

In our case all features are real numbers, so we model them with $n$ independent Gaussian distributions or alternatively a single $n$-dimensional Gaussian with diagonal covariance matrix.

In [3]:
class GaussianNaiveBayes(BaseEstimator, ClassifierMixin):
    def fit(self, X, y):
        n, m = X.shape
        self.classes_, y, self.prior_ = np.unique(y, return_inverse=True, return_counts=True)
        self.prior_ = self.prior_ / self.prior_.sum()
        
        N = len(self.classes_)
        self.mu_ = np.empty((N, m))
        self.sigma2_ = np.empty((N, m))
        
        for i in range(N):
            self.mu_[i] = X[y == i].mean(axis=0)
            self.sigma2_[i] = X[y == i].var(axis=0)
            
    def predict(self, X):
        p = self.predict_proba(X)
        
        return self.classes_[np.argmax(p, axis=1)]
    
    def predict_proba(self, X):
        N = len(self.classes_)
        m = self.mu_.shape[-1]
        p = np.empty((len(X), N))
        
        for i in range(N):
            p[:, i] = self.prior_[i] * np.exp(-1/2 * ((X - self.mu_[i])**2 / self.sigma2_[i]).sum(axis=1)) / np.sqrt((2 * np.pi)**m * np.prod(self.sigma2_[i]))
            
        p = p / p.sum(axis=1, keepdims=True)
        
        return p

In [4]:
cross_val_score(GaussianNaiveBayes(), X, y).mean()

0.93423202614379086